In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
# 테이블 형식의 데이터를 처리하는 pandas 라이브러리를 임포트
import pandas as pd

In [48]:
# train.csv, test.csv 파일을 DataFrame 형태로 읽어옵니다.
train_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/test.csv')
print('train shape: ', train_df.shape)
print('test shape: ', test_df.shape)

train shape:  (9375, 10)
test shape:  (4018, 9)


In [49]:
# train_df에서 체지방율 열을 pop 하여 y_train에 저장합니다.
y_train = train_df.pop('body fat_%')

In [50]:
# 베이스라인 코드에서는 성별(카테고리 데이터)을 제외하고 수치 데이터만 활용해 봅니다.
# 학습 및 평가 셋에 각각 성별(gender)을 제거해줍니다.
train_df['gender'] = train_df['gender'].map({'M':0, 'F':1})
test_df['gender'] = test_df['gender'].map({'M':0, 'F':1})

#정규화를 위해서 gender field 분리
train_gtemp = train_df.pop('gender')
test_gtemp = test_df.pop('gender')


In [51]:
# 8개 컬럼의 수치 데이터를 정규화 하여 학습 가능한 형태로 재가공 합니다.
# 다른 정규화 방식을 찾아서 적용해보고 정규화 방식에 따라 결과가 어떻게 달라지는지 확인해봅니다.
from sklearn.preprocessing import MinMaxScaler 

# 학습 및 평가 데이터를 DataFrame 형태로 입력받아 정규화된 데이터로 변환하는 함수 
def scale(x_train, x_test):
    columns = x_train.columns

    scaler = MinMaxScaler()

    # 학습 데이터의 age ~ broad jump_cm 열에 해당하는 값을 가져와 피팅 시킵니다.
    scaler.fit(x_train)

    # 피팅된 scaler를 기준으로 학습데이터를 정규화합니다.
    x_train = scaler.transform(x_train)
    # 평가 데이터도 마찬가지로 학습 데이터로 피팅된 scaler를 기준으로 정규화를 진행합니다.
    x_test = scaler.transform(x_test)

    # DataFrame 형태로 변환하여 반환합니다.
    x_train = pd.DataFrame(columns=columns, data=x_train)
    x_test = pd.DataFrame(columns=columns, data=x_test)

    return x_train, x_test

# 학습 데이터, 평가 데이터를 정의한 scale 함수에 전달하여 정규화된 학습 및 평가 데이터를 x_train, x_test에 저장합니다.
x_train, x_test = scale(train_df, test_df)

In [52]:
x_train['gender'] = train_gtemp
x_test['gender'] = test_gtemp

x_train['gender'] = x_train['gender'].astype('bool')
x_test['gender'] = x_test['gender'].astype('bool')

In [53]:
X_train = x_train
Y_train = y_train

In [54]:
from sklearn import svm
SVM = svm.SVR(kernel='rbf', C = 6.25, gamma=1.1)

In [57]:
import lightgbm as lgb
from sklearn.model_selection import KFold

submit = pd.read_csv('/content/drive/MyDrive/datathon/dataset/submit_sample.csv')

k_fold = KFold(n_splits=5, shuffle=True)

lgb_models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_train, y_train = X_train.iloc[train_idx], Y_train.iloc[train_idx]
    x_val, y_val = X_train.iloc[val_idx], Y_train.iloc[val_idx]
    print(x_train.info())
    print(y_train.head())

    d_train = lgb.Dataset(x_train, y_train)
    d_val = lgb.Dataset(x_val, y_val)
    
    lgb_models.append(SVM.fit(x_train, y_train))

preds = []
for model in lgb_models:
    preds.append(model.predict(x_test))
len(preds)

pred = (preds[0]+preds[1]+preds[2]+preds[3]+preds[4])/5
submit['predict'] = pred

submit.head()
submit.to_csv('/content/drive/MyDrive/datathon/dataset/submit.csv', index=False)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 9374
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      7500 non-null   float64
 1   height_cm                7500 non-null   float64
 2   diastolic                7500 non-null   float64
 3   systolic                 7500 non-null   float64
 4   gripForce                7500 non-null   float64
 5   sit and bend forward_cm  7500 non-null   float64
 6   sit-ups counts           7500 non-null   float64
 7   broad jump_cm            7500 non-null   float64
 8   gender                   7500 non-null   bool   
dtypes: bool(1), float64(8)
memory usage: 534.7 KB
None
0    12.7
3    33.0
4    28.3
5    16.3
6    29.5
Name: body fat_%, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 9374
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
-

# 모델평가

In [46]:
from sklearn.metrics import mean_absolute_error
import numpy as np

In [ ]:
model_mae = {}

mae = mean_absolute_error(model_pdf['valid'], model_p["SVM"])
model_mae["SVM"] = mae
model_mae